In [1]:
import pandas as pd
import numpy as np
import time
import warnings
import duckdb
from bs4 import BeautifulSoup
import requests
import unicodedata

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

year = 2025

In [2]:
def remove_accents(text):
    if not isinstance(text, str):
        return text
    # Normalize to NFKD (decomposes characters)
    text = unicodedata.normalize('NFKD', text)
    # Encode to ASCII bytes, ignore non-ASCII characters, then decode back to str
    return text.encode('ascii', 'ignore').decode('ascii')

df_teams = pd.read_csv("../src/team_info_xref.csv")
teams = df_teams.ABV.tolist()

df = pd.DataFrame()

for team in teams:
    time.sleep(6)
    html = requests.get(f"https://www.basketball-reference.com/teams/{team}/{year + 1}.html").text
    soup = BeautifulSoup(html, "lxml")
    target_ids = ["roster"]
    target_tables = [soup.find("table", {"id": tid}) for tid in target_ids]
    df_temp = [pd.read_html(str(tbl))[0] for tbl in target_tables][0]
    df_temp['Team'] = team
    display(df_temp)
    df = pd.concat([df, df_temp])
df = df[['Team', 'Player', 'Pos']]
df["Player"] = df["Player"].apply(lambda x: x.encode("latin1").decode("utf-8"))
df['Player'] = df['Player'].apply(remove_accents)
df.to_csv(f'../tables/{year}/plyr_pos_xref.csv', index=False)

,No.,Player,Pos,Ht,Wt,Birth Date,Birth,Exp,College,Team
0,5,Cat Barber,PG,6-2,180,"July 25, 1994",us US,R,NC State,ATL
1,13,Bogdan BogdanoviÄ,SG,6-5,225,"August 18, 1992",rs RS,4,NaN,ATL
2,45,Chaundee Brown Jr.,SG,6-5,215,"December 4, 1998",us US,R,"Wake Forest, Michigan",ATL
3,15,Clint Capela,C,6-10,256,"May 18, 1994",ch CH,7,NaN,ATL
4,20,John Collins,PF,6-9,226,"September 23, 1997",us US,4,Wake Forest,ATL
5,2,Sharife Cooper,PG,6-0,176,"June 11, 2001",us US,R,Auburn,ATL
6,10,Gorgui Dieng,C,6-10,252,"January 18, 1990",sn SN,8,Louisville,ATL
7,8,Danilo Gallinari,PF,6-10,236,"August 8, 1988",it IT,12,NaN,ATL
8,14,Malcolm Hill,SG,6-6,220,"October 26, 1995",us US,R,Illinois,ATL
9,18,Solomon Hill,SF,6-6,226,"March 18, 1991",us US,8,Arizona,ATL


,No.,Player,Pos,Ht,Wt,Birth Date,Birth,Exp,College,Team
0,7,Jaylen Brown,SF,6-6,223,"October 24, 1996",us US,5,California,BOS
1,28,Bruno Fernando,C,6-9,240,"August 15, 1998",ao AO,2,Maryland,BOS
2,8,Malik Fitts,PF,6-5,230,"July 4, 1997",us US,1,"South Florida, Saint Mary's",BOS
3,13,Enes Freedom,C,6-10,250,"May 20, 1992",ch CH,10,Kentucky,BOS
4,30,Sam Hauser,SF,6-7,217,"December 8, 1997",us US,R,"Marquette, Virginia",BOS
5,41,Juancho HernangÃ³mez,PF,6-9,214,"September 28, 1995",es ES,5,NaN,BOS
6,42,Al Horford,C,6-8,240,"June 3, 1986",do DO,14,Florida,BOS
7,43,Justin Jackson,SF,6-8,220,"March 28, 1995",us US,4,UNC,BOS
8,55,Joe Johnson,SG,6-7,240,"June 29, 1981",us US,17,Arkansas,BOS
9,40,Luke Kornet,C,7-1,250,"July 15, 1995",us US,4,Vanderbilt,BOS


,No.,Player,Pos,Ht,Wt,Birth Date,Birth,Exp,College,Team
0,21,LaMarcus Aldridge,C,6-11,250,"July 19, 1985",us US,15,Texas,BRK
1,95,DeAndre' Bembry,SF,6-5,210,"July 4, 1994",us US,5,Saint Joseph's,BRK
2,1,Bruce Brown,SF,6-4,202,"August 15, 1996",us US,3,Miami (FL),BRK
3,0,Jevon Carter,PG,6-0,200,"September 14, 1995",us US,3,West Virginia,BRK
4,33,Nic Claxton,C,6-11,215,"April 17, 1999",us US,2,Georgia,BRK
5,30,Seth Curry,SG,6-1,185,"August 23, 1990",us US,7,"Liberty, Duke",BRK
6,9,Goran DragiÄ,PG,6-4,190,"May 6, 1986",si SI,13,NaN,BRK
7,"0, 4",Andre Drummond,C,6-11,279,"August 10, 1993",us US,9,UConn,BRK
8,6,David Duke Jr.,PG,6-4,204,"October 13, 1999",us US,R,Providence,BRK
9,7,Kevin Durant,PF,6-11,240,"September 29, 1988",us US,13,Texas,BRK


,No.,Player,Pos,Ht,Wt,Birth Date,Birth,Exp,College,Team
0,2,LaMelo Ball,PG,6-7,180,"August 22, 2001",us US,1,NaN,CHO
1,5,James Bouknight,SG,6-4,190,"September 18, 2000",us US,R,UConn,CHO
2,0,Miles Bridges,PF,6-7,225,"March 21, 1998",us US,3,Michigan State,CHO
3,22,Vernon Carey Jr.,C,6-10,270,"February 25, 2001",us US,1,Duke,CHO
4,8,Montrezl Harrell,C,6-7,240,"January 26, 1994",us US,6,Louisville,CHO
5,20,Gordon Hayward,SF,6-7,225,"March 23, 1990",us US,11,Butler,CHO
6,23,Kai Jones,C,6-11,221,"January 19, 2001",bs BS,R,Texas,CHO
7,98,Arnoldas Kulboka,SF,6-10,220,"January 4, 1998",lt LT,R,NaN,CHO
8,16,Scottie Lewis,SG,6-4,188,"March 12, 2000",us US,R,Florida,CHO
9,11,Cody Martin,SF,6-6,205,"September 28, 1995",us US,2,"NC State, Nevada",CHO


,No.,Player,Pos,Ht,Wt,Birth Date,Birth,Exp,College,Team
0,2,Lonzo Ball,PG,6-5,190,"October 27, 1997",us US,4,UCLA,CHI
1,20,Jordan Bell,C,6-8,216,"January 7, 1995",us US,4,Oregon,CHI
2,13,Tony Bradley,C,6-10,248,"January 8, 1998",us US,4,UNC,CHI
3,7,Troy Brown Jr.,SF,6-7,215,"July 28, 1999",us US,3,Oregon,CHI
4,6,Alex Caruso,SG,6-5,186,"February 28, 1994",us US,4,Texas A&M,CHI
5,25,Tyler Cook,PF,6-8,255,"September 23, 1997",us US,2,Iowa,CHI
6,11,DeMar DeRozan,PF,6-6,220,"August 7, 1989",us US,12,USC,CHI
7,12,Ayo Dosunmu,SG,6-4,200,"January 17, 2000",us US,R,Illinois,CHI
8,3,Devon Dotson,PG,6-1,185,"August 2, 1999",us US,1,Kansas,CHI
9,24,Javonte Green,SF,6-5,205,"July 23, 1993",us US,2,Radford,CHI


...

,No.,Player,Pos,Ht,Wt,Birth Date,Birth,Exp,College,Team
0,35,Marvin Bagley III,PF,6-10,235,"March 14, 1999",us US,3,Duke,SAC
1,40,Harrison Barnes,PF,6-7,225,"May 30, 1992",us US,9,UNC,SAC
2,3,Terence Davis,SG,6-4,201,"May 16, 1997",us US,2,Ole Miss,SAC
3,0,Donte DiVincenzo,SG,6-4,203,"January 31, 1997",us US,3,Villanova,SAC
4,5,De'Aaron Fox,PG,6-3,185,"December 20, 1997",us US,4,Kentucky,SAC
5,0,Tyrese Haliburton,SG,6-5,185,"February 29, 2000",us US,1,Iowa State,SAC
6,8,Maurice Harkless,SF,6-7,220,"May 11, 1993",us US,9,St. John's,SAC
7,24,Buddy Hield,SG,6-4,220,"December 17, 1992",bs BS,5,Oklahoma,SAC
8,9,Justin Holiday,SG,6-6,180,"April 5, 1989",us US,8,Washington,SAC
9,22,Richaun Holmes,C,6-9,235,"October 15, 1993",us US,6,Bowling Green,SAC


,No.,Player,Pos,Ht,Wt,Birth Date,Birth,Exp,College,Team
0,31,Keita Bates-Diop,SF,6-8,229,"January 23, 1996",us US,3,Ohio State,SAS
1,18,Devontae Cacok,PF,6-7,240,"October 8, 1996",us US,2,UNC Wilmington,SAS
2,23,Zach Collins,C,6-9,250,"November 19, 1997",us US,3,Gonzaga,SAS
3,14,Drew Eubanks,C,6-10,245,"February 1, 1997",us US,3,Oregon State,SAS
4,7,Bryn Forbes,SG,6-2,205,"July 23, 1993",us US,5,Michigan State,SAS
5,41,Juancho HernangÃ³mez,PF,6-9,214,"September 28, 1995",es ES,5,NaN,SAS
6,3,Keldon Johnson,SF,6-6,220,"October 11, 1999",us US,2,Kentucky,SAS
7,28,Tyler Johnson,PG,6-3,186,"May 7, 1992",us US,7,Fresno State,SAS
8,33,Tre Jones,PG,6-1,185,"January 8, 2000",us US,1,Duke,SAS
9,35,Anthony Lamb,SF,6-6,227,"January 20, 1998",us US,1,University of Vermont,SAS


,No.,Player,Pos,Ht,Wt,Birth Date,Birth,Exp,College,Team
0,5,Precious Achiuwa,C,6-8,243,"September 19, 1999",ng NG,1,Memphis,TOR
1,3,OG Anunoby,SF,6-7,240,"July 17, 1997",gb GB,4,Indiana,TOR
2,45,Dalano Banton,PG,6-8,203,"November 7, 1999",ca CA,R,Nebraska,TOR
3,4,Scottie Barnes,PF,6-8,237,"August 1, 2001",us US,R,Florida State,TOR
4,24,Khem Birch,C,6-8,233,"September 28, 1992",ca CA,4,"Pitt, UNLV",TOR
5,17,Isaac Bonga,SF,6-8,180,"November 8, 1999",de DE,3,NaN,TOR
6,25,Chris Boucher,SF,6-8,200,"January 11, 1993",lc LC,4,"New Mexico Junior College, Northwest College, ...",TOR
7,1,Armoni Brooks,PG,6-3,195,"June 5, 1998",us US,1,Houston,TOR
8,11,Justin Champagnie,SF,6-6,206,"June 29, 2001",us US,R,Pitt,TOR
9,8,Sam Dekker,PF,6-9,230,"May 6, 1994",us US,4,Wisconsin,TOR


,No.,Player,Pos,Ht,Wt,Birth Date,Birth,Exp,College,Team
0,6,Nickeil Alexander-Walker,SG,6-5,205,"September 2, 1998",ca CA,2,Virginia Tech,UTA
1,20,Udoka Azubuike,C,6-11,270,"September 17, 1999",ng NG,1,Kansas,UTA
2,44,Bojan BogdanoviÄ,PF,6-7,226,"April 18, 1989",ba BA,7,NaN,UTA
3,13,Jared Butler,SG,6-3,193,"August 25, 2000",us US,R,Baylor,UTA
4,6,Zylan Cheatham,SF,6-5,220,"November 17, 1995",us US,1,"San Diego State, Arizona State",UTA
5,0,Jordan Clarkson,SG,6-5,194,"June 7, 1992",us US,7,"Tulsa, Missouri",UTA
6,11,Mike Conley,PG,6-1,175,"October 11, 1987",us US,14,Ohio State,UTA
7,24,Malik Fitts,PF,6-5,230,"July 4, 1997",us US,1,"South Florida, Saint Mary's",UTA
8,3,Trent Forrest,PG,6-4,210,"June 12, 1998",us US,1,Florida State,UTA
9,8,Rudy Gay,PF,6-8,250,"August 17, 1986",us US,15,UConn,UTA


,No.,Player,Pos,Ht,Wt,Birth Date,Birth,Exp,College,Team
0,9,Deni Avdija,SF,6-8,228,"January 3, 2001",il IL,1,NaN,WAS
1,17,Joel Ayayi,SG,6-4,180,"March 5, 2000",fr FR,R,Gonzaga,WAS
2,3,Bradley Beal,SG,6-4,207,"June 28, 1993",us US,9,Florida,WAS
3,42,DÄvis BertÄns,PF,6-10,225,"November 12, 1992",lv LV,5,NaN,WAS
4,13,Thomas Bryant,C,6-9,248,"July 31, 1997",us US,4,Indiana,WAS
5,1,Kentavious Caldwell-Pope,SG,6-5,204,"February 18, 1993",us US,8,Georgia,WAS
6,22,Vernon Carey Jr.,C,6-10,270,"February 25, 2001",us US,1,Duke,WAS
7,26,Spencer Dinwiddie,PG,6-5,215,"April 6, 1993",us US,7,Colorado,WAS
8,12,Jaime Echenique,C,6-11,258,"April 27, 1997",co CO,R,"Trinity Valley CC, Wichita State",WAS
9,21,Daniel Gafford,C,6-10,265,"October 1, 1998",us US,2,Arkansas,WAS


# Update NULL Positions

In [14]:
df = pd.read_csv(f"../tables/{year}/season_gamelogs.csv")
print('Null Pos Count:', df[(df.Pos.isnull())].shape[0])
df2 = pd.read_csv(f'../tables/{year}/plyr_pos_xref.csv')
orig_cols = df.columns
df = pd.merge(df, df2, how = 'left', on = ['Team', 'Player'], suffixes=('', '_y'))
df['Pos'] = df.Pos.fillna(df.Pos_y)
print('Null Pos Count:', df[(df.Pos.isnull())].shape[0])
df = df[orig_cols]
df.to_csv(f"../tables/{year}/season_gamelogs.csv", index=False)

Null Pos Count: 32
Null Pos Count: 32
